## <center> CNN applied on RGBD images </center>

In [1]:
#import necessary libraries
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from scipy.ndimage import zoom
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
data_dir = r'C:\Users\Siwar\Downloads\data'  
indoor_dir = os.path.join(data_dir, 'indoors')  # Path to the indoor images and depth maps folder
outdoor_dir = os.path.join(data_dir, 'outdoors')  # Path to the outdoor images and depth maps folder

# Function to read and process the images and depth maps
def read_data(directory):
    images = []
    depth_maps = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith('.png'):
            img = Image.open(file_path)  
            images.append(img)
        elif filename.endswith('.npy'):
            depth_map = np.load(file_path)  
            depth_maps.append(depth_map)
    return images, depth_maps

# Read indoor images and depth maps
indoor_images, indoor_depth_maps = read_data(indoor_dir)
print(f"Total indoor images: {len(indoor_images)}")
print(f"Total indoor depth maps: {len(indoor_depth_maps)}")

# Read outdoor images and depth maps
outdoor_images, outdoor_depth_maps = read_data(outdoor_dir)
print(f"Total outdoor images: {len(outdoor_images)}")
print(f"Total outdoor depth maps: {len(outdoor_depth_maps)}")


Total indoor images: 829
Total indoor depth maps: 829
Total outdoor images: 834
Total outdoor depth maps: 834


In [3]:
# Resize indoor images
resized_indoor_images = []
for image in indoor_images:
    resized_image = image.resize((64, 48))
    resized_indoor_images.append(resized_image)

# Resize outdoor images
resized_outdoor_images = []
for image in outdoor_images:
    resized_image = image.resize((64, 48))
    resized_outdoor_images.append(resized_image)

In [4]:
# Resize indoor depth maps
resized_indoor_depth_maps = []
for depth_map in indoor_depth_maps:
    depth_map = depth_map.astype(np.uint8)
    resized_depth_map = zoom(depth_map, (1/16, 1/16, 1), order=1)
    resized_depth_map = resized_depth_map.astype(np.uint8)
    resized_indoor_depth_maps.append(resized_depth_map)

# Resize outdoor depth maps
resized_outdoor_depth_maps = []
for depth_map in outdoor_depth_maps:
    depth_map = depth_map.astype(np.uint8)
    resized_depth_map = zoom(depth_map, (1/16, 1/16, 1), order=1)
    resized_depth_map = resized_depth_map.astype(np.uint8)
    resized_outdoor_depth_maps.append(resized_depth_map)

In [5]:
# Convert resized RGB images to TensorFlow tensors
indoor_rgb_images_tensor = tf.stack([tf.image.convert_image_dtype(image, tf.float32) for image in resized_indoor_images])
outdoor_rgb_images_tensor = tf.stack([tf.image.convert_image_dtype(image, tf.float32) for image in resized_outdoor_images])

# Convert resized depth maps to TensorFlow tensors
indoor_depth_maps_tensor = tf.stack([tf.image.convert_image_dtype(depth_map, tf.float32) for depth_map in resized_indoor_depth_maps])
outdoor_depth_maps_tensor = tf.stack([tf.image.convert_image_dtype(depth_map, tf.float32) for depth_map in resized_outdoor_depth_maps])

# Concatenate RGB images and depth maps along the channel axis (axis=3) to create RGBD images
indoor_rgbd_images_tensor = tf.concat([indoor_rgb_images_tensor, indoor_depth_maps_tensor], axis=3)
outdoor_rgbd_images_tensor = tf.concat([outdoor_rgb_images_tensor, outdoor_depth_maps_tensor], axis=3)


In [6]:
len(indoor_rgbd_images_tensor), len(outdoor_rgbd_images_tensor)

(829, 834)

In [7]:
indoor_rgbd_images_tensor[0].shape

TensorShape([48, 64, 4])

In [8]:
indoor_rgbd_images_numpy = indoor_rgbd_images_tensor.numpy()
outdoor_rgbd_images_numpy = outdoor_rgbd_images_tensor.numpy()

# Split indoor RGBD images into train, validation and test sets
indoor_train_rgbd, indoor_test_rgbd = train_test_split(indoor_rgbd_images_numpy, test_size=0.2, random_state=43)
indoor_train_rgbd, indoor_val_rgbd = train_test_split(indoor_train_rgbd, test_size=0.2, random_state=43)
# Split outdoor RGBD images into train, validation and test sets
outdoor_train_rgbd, outdoor_test_rgbd = train_test_split(outdoor_rgbd_images_numpy, test_size=0.2, random_state=43)
outdoor_train_rgbd, outdoor_val_rgbd = train_test_split(outdoor_train_rgbd, test_size=0.1, random_state=43)

In [9]:
len(indoor_train_rgbd),len(indoor_val_rgbd),len(indoor_test_rgbd), len(outdoor_train_rgbd),len(outdoor_val_rgbd),len(outdoor_test_rgbd)

(530, 133, 166, 600, 67, 167)

In [10]:
# train, validation and test data
train_data=np.concatenate((indoor_train_rgbd, outdoor_train_rgbd), axis=0)
val_data=np.concatenate((indoor_val_rgbd, outdoor_val_rgbd), axis=0)
test_data=np.concatenate((indoor_test_rgbd, outdoor_test_rgbd), axis=0)
train_labels = np.concatenate((np.zeros(len(indoor_train_rgbd)), np.ones(len(outdoor_train_rgbd))))
test_labels = np.concatenate((np.zeros(len(indoor_test_rgbd)), np.ones(len(outdoor_test_rgbd))))
val_labels = np.concatenate((np.zeros(len(indoor_val_rgbd)), np.ones(len(outdoor_val_rgbd))))
len(train_data),len(val_data),len(test_data)

(1130, 200, 333)

In [11]:
np.random.seed(43)
tf.random.set_seed(43)
# the model architecture
model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(48, 64, 4), kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [12]:
model.fit(train_data, train_labels, epochs=30, batch_size=32, validation_data=(val_data, val_labels))


Epoch 1/30
36/36 [==============================] - 18s 113ms/step - loss: 3.1766 - accuracy: 0.8088 - val_loss: 3.5502 - val_accuracy: 0.6650
Epoch 2/30
36/36 [==============================] - 4s 112ms/step - loss: 2.5751 - accuracy: 0.8628 - val_loss: 3.4262 - val_accuracy: 0.6650
Epoch 3/30
36/36 [==============================] - 4s 118ms/step - loss: 2.1262 - accuracy: 0.9044 - val_loss: 3.0510 - val_accuracy: 0.6650
Epoch 4/30
36/36 [==============================] - 4s 104ms/step - loss: 1.8249 - accuracy: 0.9221 - val_loss: 2.8001 - val_accuracy: 0.6650
Epoch 5/30
36/36 [==============================] - 4s 103ms/step - loss: 1.5585 - accuracy: 0.9230 - val_loss: 2.4563 - val_accuracy: 0.6650
Epoch 6/30
36/36 [==============================] - 4s 109ms/step - loss: 1.3533 - accuracy: 0.9310 - val_loss: 2.2682 - val_accuracy: 0.6650
Epoch 7/30
36/36 [==============================] - 4s 104ms/step - loss: 1.1927 - accuracy: 0.9434 - val_loss: 1.8584 - val_accuracy: 0.6700
Epoch

In [13]:
predictions = model.predict(test_data)

11/11 [==============================] - 1s 26ms/step


In [14]:
# Calculate accuracy on train data
train_loss, train_accuracy = model.evaluate(train_data, train_labels)

# Calculate accuracy on validation data
val_loss, val_accuracy = model.evaluate(val_data, val_labels)

print("Train Accuracy:", round(train_accuracy,2))
print("Validation Accuracy:", round(val_accuracy,2))
print("train_loss:",round(train_loss,2))
print("val_loss:",round(train_loss,2))

7/7 [==============================] - 0s 27ms/step - loss: 0.3362 - accuracy: 0.9600
Train Accuracy: 0.98
Validation Accuracy: 0.96
train_loss: 0.29
val_loss: 0.29


In [15]:
predicted_labels = (predictions > 0.5).astype(int)
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels)
recall = recall_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels)
auc_roc = roc_auc_score(test_labels, predictions)
print("Accuracy:", round(accuracy, 2))
print("Precision:", round(precision, 2))
print("Recall:", round(recall, 2))
print("F1 Score:", round(f1, 2))
print("AUC-ROC Score:", round(auc_roc, 2))

Accuracy: 0.95
Precision: 0.91
Recall: 0.99
F1 Score: 0.95
AUC-ROC Score: 0.99
